# Multi-class LDA

In [1]:
import numpy as np
import pandas as pd
from scipy.stats import multivariate_normal as m_norm
from IPython.display import display, clear_output #for HTML printing
import timeit

In [2]:
x_test_df = pd.read_csv("X_test.csv")
x_train_df = pd.read_csv("X_train.csv")
y_train_df = pd.read_csv("Y_train.csv")

#display(x_train_df.head())
#display(x_train_df.info())
#display(x_train_df.describe())
#display(y_train_df.head())

# creating a map {surface : numerical class}
class_namelist = y_train_df.surface.unique()
class_dict = {x:y for x,y in zip(range(0, len(class_namelist)), class_namelist)}
inv_class_dict = {v: k for k, v in class_dict.items()}

display(class_dict)
#display(inv_class_dict)

# adding 'surface' (response variable) to x_train dataframe and lets call it train_df
x_train_df = x_train_df.merge(y_train_df, on="series_id", how="left")
train_df = x_train_df    # don't worry about memory, its just a pointer


# adding 'class' (numerical class) to train_df
train_df['class'] = train_df['surface'].apply(lambda x: inv_class_dict[x]) 
display(train_df.tail())

{0: 'fine_concrete',
 1: 'concrete',
 2: 'soft_tiles',
 3: 'tiled',
 4: 'soft_pvc',
 5: 'hard_tiles_large_space',
 6: 'carpet',
 7: 'hard_tiles',
 8: 'wood'}

,row_id,series_id,measurement_number,orientation_X,orientation_Y,orientation_Z,orientation_W,angular_velocity_X,angular_velocity_Y,angular_velocity_Z,linear_acceleration_X,linear_acceleration_Y,linear_acceleration_Z,group_id,surface,class
487675,3809_123,3809,123,0.62871,-0.76878,-0.084391,0.081093,0.003167,0.093760,-0.14274,3.27180,2.0115,-9.0063,56,soft_pvc,4
487676,3809_124,3809,124,0.62884,-0.76868,-0.084365,0.081099,0.014994,0.032637,-0.13238,4.42750,3.0696,-8.1257,56,soft_pvc,4
487677,3809_125,3809,125,0.62891,-0.76861,-0.084345,0.081178,-0.031184,-0.003961,-0.13894,2.70480,4.2622,-8.1443,56,soft_pvc,4
487678,3809_126,3809,126,0.62903,-0.76850,-0.084414,0.081231,-0.069153,0.013229,-0.13021,2.54100,4.7130,-9.4435,56,soft_pvc,4
487679,3809_127,3809,127,0.62915,-0.76839,-0.084441,0.081284,-0.042769,0.034049,-0.12580,0.82391,4.2751,-10.4980,56,soft_pvc,4


In [3]:
# all features
feature_list = train_df.loc[:, 'orientation_X':'linear_acceleration_Z'].columns.values.tolist()

# x_train_df.mean(axis=0) # this is slower because of object column
# x_train_df._get_numeric_data().mean(axis=0) # either use this or better

# calculating class-wise sample means
sample_mean = [0]*len(feature_list)   # for each feature
sample_size = [0]*len(class_dict)  # size for each class
sample_cov_dfs = [0]*len(class_dict)  # covariance for each class ( each item will be a df )

sample_means = [sample_mean for i in range(0, len(class_dict))] # a list of arrays (each array is for each class)

# for each class in class_dict
for idx, surface_i in enumerate(class_dict.values()):
    temp_df = train_df.loc[train_df.surface == surface_i]
    sample_size_i = len(temp_df.index) # this gets the row count of the dataframe
    
    temp_df2 = temp_df.loc[:, 'orientation_X':'linear_acceleration_Z']
    
    sample_means[idx] = temp_df2.mean(axis=0).values # store mean into respective array
    sample_cov_dfs[idx] = temp_df2.cov()
    sample_size[idx] = sample_size_i # store sample size into a list
    

In [4]:
#display(sample_means)
sample_mean_dict = dict(zip(class_dict.values(), sample_means))
display(sample_mean_dict)
display(sample_size)

# compute prevalences
# P(Y = class) = sample_size_class/sample_size_training

sample_size_all = len(train_df.index)

prevalence = [class_size/sample_size_all for class_size in sample_size]
print(prevalence[0])
#display(prevalence) #should sum to 1.0

#display(sample_cov_dfs[0])
#display(sample_cov_dfs[1])
#df = pd.DataFrame()
#df = pd.concat([sample_cov_dfs[0], sample_cov_dfs[1]])
#display(df)

assert sum(sample_size) == len(train_df.index)

{'fine_concrete': array([-1.43290155e-01,  1.88151457e-01,  2.85006496e-02, -2.16831563e-02,
        -9.91504906e-05,  3.16736093e-03, -2.44271486e-03,  1.36297485e-01,
         2.92210215e+00, -9.35607003e+00]),
 'concrete': array([-2.07606983e-01,  7.89323534e-02,  1.03509340e-02, -3.21066444e-02,
         9.25171794e-04,  2.18164319e-02, -6.27181845e-02,  1.11593092e-01,
         2.90773270e+00, -9.35640755e+00]),
 'soft_tiles': array([ 2.76689163e-01,  1.77857122e-02,  4.52426477e-03,  4.20254501e-02,
         6.40250325e-04,  1.75021390e-02, -4.89125478e-02,  8.41187577e-02,
         2.91412359e+00, -9.35947358e+00]),
 'tiled': array([-2.75754295e-01,  7.50029355e-02,  1.08687175e-02, -4.20898577e-02,
         1.01947216e-04,  6.40349476e-03, -1.26683436e-02,  1.35559725e-01,
         2.91380343e+00, -9.35704482e+00]),
 'soft_pvc': array([ 3.09570136e-01, -1.14566104e-02,  5.43322975e-03,  4.33312302e-02,
         6.92125374e-04,  1.48694477e-02, -4.03870933e-02,  1.28838871e-01,


[46464, 99712, 38016, 65792, 93696, 39424, 24192, 2688, 77696]

0.09527559055118111


P(Y=class|x) = P(x|Y=class).P(Y=class)

P(x|Y=class) = normal dist. pdf(x|class parameters)

P(Y=class) = prevalence

The classifier is given by:

classifier(x) = argmax {P(Y=i|x)} (the arg being 'i', the class)

In [5]:
#%%timeit
start = timeit.default_timer()
for class_num, class_name in enumerate(class_dict.values()):
    class_mean = sample_mean_dict[class_name]
    class_cov = sample_cov_dfs[class_num]
    pdf = m_norm.pdf(train_df[feature_list].values, mean=class_mean, cov=class_cov)
    preval = prevalence[class_num]
    posterior_prob = preval*pdf
    arg = class_num
    string_t = "prob_class" + str(class_num)
    train_df[string_t] = posterior_prob

stop = timeit.default_timer()

elapsed = stop - start
print('{:.1f} ms'.format(elapsed*1000))

train_df['prediction'] = train_df[['prob_class0', 'prob_class1', 'prob_class2', 'prob_class3', 'prob_class4', 'prob_class5', 'prob_class6', 'prob_class7', 'prob_class8']].values.argmax(axis=1)
train_df['prediction'].replace(class_dict, inplace=True)

error_count = sample_size_all - sum(train_df['surface'] == train_df['prediction'])

print('Error count = {}'.format(error_count))
print('Training error = {:.2f}%'.format(error_count/sample_size_all))

1368.7 ms
Error count = 254349
Training error = 0.52%


In [6]:
train_df.to_csv("out.csv")